In [1]:
import pandas as pd
import sys
sys.path.append('../../utils')  # go up two folders, then into utils

from data_cleaning_utils import (
    data_cleaning,
    enrich_invoice_flags,
    uom_cleaning,
    flag_fully_converted_invoices,
    add_freight_per_invoice,
    filter_valid_invoices,
    increase_sample_size,
)  # or whatever functions you have


ImportError: cannot import name 'data_cleaning' from 'data_cleaning_utils' (c:\Users\ITafr\OneDrive\Desktop\Diverzify\diverzify_freight_analysis\notebooks\nigel\../../utils\data_cleaning_utils.py)

In [ ]:
df = pd.read_csv('../../data/input/Freight_Cost_Analysis_CY2024-03.25.csv',encoding="latin1")  # replace with your actual data loading method

C:\Users\ITafr\AppData\Local\Temp\ipykernel_29608\1107790926.py:1: DtypeWarning: Columns (12) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('../../data/input/Freight_Cost_Analysis_CY2024-03.25.csv',encoding="latin1")  # replace with your actual data loading method


In [ ]:
conversion_csv_path = "../../data/input/freight_model/conversion_table_standardized.csv"


In [ ]:
df = data_cleaning(df,base_path='../../data/input')
df = uom_cleaning(df)
df = flag_fully_converted_invoices(df, conversion_csv_path)
df = enrich_invoice_flags(df)
df = add_freight_per_invoice(df)
df = increase_sample_size(df)
df.head(2)

INFO:root:🔧 Running data_cleaning...
INFO:root:✅ data_cleaning complete.
INFO:root:✅ fixing unit of measure.
INFO:root:✅ Flagging fully converted invoices.
INFO:root:✅ Enriching invoice data.


✅ 8590 invoices have all account == 2008 rows with valid conversion codes
Number of invoices with at least one freight line: 55942


c:\Users\ITafr\OneDrive\Desktop\Diverzify\diverzify_freight_analysis\notebooks\nigel\../../utils\data_cleaning_utils.py:216: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  freight_count > 1).fillna(False)
c:\Users\ITafr\OneDrive\Desktop\Diverzify\diverzify_freight_analysis\notebooks\nigel\../../utils\data_cleaning_utils.py:226: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  component_count > 1).fillna(False)


Number of invoices with multiple freight lines: 197


c:\Users\ITafr\OneDrive\Desktop\Diverzify\diverzify_freight_analysis\notebooks\nigel\../../utils\data_cleaning_utils.py:236: FutureWarning: Downcasting object dtype arrays on .fillna, .ffill, .bfill is deprecated and will change in a future version. Call result.infer_objects(copy=False) instead. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  commodity_count > 1).fillna(False)


...

INFO:root:✅ Calculating freight per invoice.


Number of invoices where at least one ACCOUNT == 2008 has Priority == 'Yes': 23536


INFO:root:✅ Completed adding freight to invoice.
INFO:root:✅ increasing freight per invoice.
INFO:root:✅ completed increasing sample size.


0     0.000000
1    55.625259
2     0.000000
3     0.000000
4     0.000000
Name: pct_priority_product_invoice_total, dtype: float64
0    False
1    False
2    False
3    False
4    False
Name: pct_priority_product_invoice_total, dtype: bool


,site,site_description,supplier_no,supplier_name,invoice_id,invoice_no,date_posted,project_id,project_name,account,...,all_accounts_2008_uom_classified,all_2008_accounts_converted,has_freight_line,multiple_freight_lines,multiple_parts,multiple_commodities,all__invoice_priority_products_(2008),any__invoice_priority_products_(2008),freight_per_invoice,low_mix_priority_flag
0,DIT,Diverzify Itasca,121550,Mohawk Industries,731977,C2665159,25-Jun-24,2312127222,Blue Chip Casino Deli,5504,...,True,True,True,False,False,False,True,True,19.07,True
1,SPN,Spectra Norcross,103277,William M. Bird,531030,656881,01-Mar-24,2401132763,FAROPOINT LOBBY RENOVATION,2008,...,True,True,True,False,False,False,True,True,5.79,True


In [ ]:
# export the cleaned DataFrame to a CSV file
df.to_csv('../../data/output/cleaned_freight_data.csv', index=False)

In [ ]:
df['classification'].unique()

array(['No UOM', 'Classified', 'Unclassified'], dtype=object)

### Begin Funnel 

In [ ]:
# How many distinct invoices 
distinct_invoices = df.groupby('invoice_id').ngroups
print(f"Distinct number of invoices: {distinct_invoices}")

Distinct number of invoices: 188087


In [ ]:
# Invoices with project ID 
unique_invoices_with_project_id = df[df['project_id'].notnull()]['invoice_id'].nunique()
print(f"Unique invoices with project ID: {unique_invoices_with_project_id}")

Unique invoices with project ID: 161241


In [ ]:
# Invoices with PO
invoices_with_po = df[df['project_id'].notnull() & df['po_no'].notnull()]['invoice_id'].nunique()
print(f"Invoices with project ID and PO numbers: {invoices_with_po}")

Invoices with project ID and PO numbers: 152548


In [ ]:
df = df[df['invoice_id'].isin(df[df['po_no'].notnull()]['invoice_id'])]
df.shape

(406310, 48)

In [ ]:
df.columns

Index(['site', 'site_description', 'supplier_no', 'supplier_name',
       'invoice_id', 'invoice_no', 'date_posted', 'project_id', 'project_name',
       'account', 'account_description', 'planned_delivery_date',
       'ship_to_zip', 'po_no', 'po_line_no', 'po_rel_no', 'receipt_no',
       'part_no', 'part_description', 'comm_1', 'comm_2', 'po_purch_qty',
       'purch_uom', 'po_inv_qty', 'inv_uom', 'invoiced_line_qty',
       'invoice_line_total', 'po_price', 'commodity_group', 'description',
       'old/new', 'priority', 'match_commodity', 'match_supplier',
       'classification', 'new_commodity_description', 'new_commodity_group',
       'conversion_code', 'all_accounts_2008_uom_classified',
       'all_2008_accounts_converted', 'has_freight_line',
       'multiple_freight_lines', 'multiple_parts', 'multiple_commodities',
       'all__invoice_priority_products_(2008)',
       'any__invoice_priority_products_(2008)', 'freight_per_invoice',
       'low_mix_priority_flag'],
      dty

In [ ]:
unique_invoices_with_freight = df[df['has_freight_line'] == True]['invoice_id'].nunique()
print(f"Unique invoices with freight line: {unique_invoices_with_freight}")

Unique invoices with freight line: 52316


In [ ]:
unique_invoices_with_freight_and_single_line = df[(df['has_freight_line'] == True) & (df['multiple_freight_lines'] == False)]['invoice_id'].nunique()
print(f"Unique invoices with freight line and no multiple freight lines: {unique_invoices_with_freight_and_single_line}")

Unique invoices with freight line and no multiple freight lines: 52122


In [ ]:
filtered_invoices = df[
    (df['has_freight_line'] == True) &
    (df['multiple_freight_lines'] == False) &
    (df['any__invoice_priority_products_(2008)'] == True)
]['invoice_id'].nunique()

print(f"Unique invoices with the specified conditions: {filtered_invoices}")

Unique invoices with the specified conditions: 18511


In [ ]:
unique_invoices = df[
    (df['has_freight_line'] == True) &
    (df['multiple_freight_lines'] == False) &
    (df['any__invoice_priority_products_(2008)'] == True) &
    (df['low_mix_priority_flag'] == True)
]['invoice_id'].nunique()

print(f"Unique invoices with the specified conditions: {unique_invoices}")

Unique invoices with the specified conditions: 17976


In [ ]:
# How much freight cost is there in the invoices with project ID and PO numbers?
sum_invoice_line_total = df[
    (df['invoice_id'].isin(df[
        (df['has_freight_line'] == True) &
        (df['multiple_freight_lines'] == False) &
        (df['any__invoice_priority_products_(2008)'] == True) &
        (df['low_mix_priority_flag'] == True)
    ]['invoice_id'])) &
    (df['account'] == 5504)
]['invoice_line_total'].sum()

print(f"Sum of invoice_line_total for unique invoices where account == 2008: {sum_invoice_line_total}")

Sum of invoice_line_total for unique invoices where account == 2008: 6265994.24


In [ ]:
unique_invoices_with_uom = df[
    (df['has_freight_line'] == True) &
    (df['multiple_freight_lines'] == False) &
    (df['any__invoice_priority_products_(2008)'] == True) &
    (df['low_mix_priority_flag'] == True) &
    (df['inv_uom'].isin(['SQYD', 'SQFT']))
]['invoice_id'].nunique()

print(f"Unique invoices with specified conditions and inv_uom as SQYD or SQFT: {unique_invoices_with_uom}")

Unique invoices with specified conditions and inv_uom as SQYD or SQFT: 16372


In [ ]:
um_invoice_line_total = df[
    (df['invoice_id'].isin(df[
        (df['has_freight_line'] == True) &
        (df['multiple_freight_lines'] == False) &
        (df['any__invoice_priority_products_(2008)'] == True) &
        (df['low_mix_priority_flag'] == True) &
        (df['inv_uom'].isin(['SQYD', 'SQFT']))
    ]['invoice_id'])) &
    (df['account'] == 5504)
]['invoice_line_total'].sum()

um_invoice_line_total

5718669.33